In [45]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from enum import Enum
from datetime import datetime
from flask import Flask, render_template, request


In [54]:
# create an enum for importance
class Importance(Enum):
    NONE = 'None'
    KINDA_IMPORTANT = 'Kinda Important'
    SUPER_IMPORTANT = 'Super Important'
    DATE_DEPENDENT = 'Date Dependent'

# create an enum for overdue
class IfOverdue(Enum):
    MARK_RED_PRIORITIZE = 'Mark Red and Prioritize'
    MARK_RED = 'Mark Red'
    NEVER_MIND = 'Never Mind'

In [47]:
# create class Task
class Task:
    def __init__(self, task, is_done=False, label=None, date_due=None, importance=None, is_prioritized=False, is_overdue=False,
                 if_overdue=None, details=None, steps=None, thoughts=None):
        self.task = task
        self.is_done = is_done
        self.label=label
        self.date_added = datetime.now()  # Automatically set to the current date and time
        self.date_due = date_due
        # Check if importance is a valid member of the Importance enum
        if importance and importance not in Importance.__members__:
            raise ValueError(f"{importance} is not a valid Importance")
        self.importance = Importance[importance] if importance else None
        self.is_prioritized = is_prioritized


        # do-it-if-have-time attributes:
        self.is_overdue = is_overdue
        # Check if if_overdue is a valid member of the IfOverdue enum
        if if_overdue and if_overdue not in IfOverdue.__members__:
            raise ValueError(f"{if_overdue} is not a valid IfOverdue")
        self.if_overdue = IfOverdue[if_overdue] if if_overdue else None        
        self.details = details 
        self.steps = steps
        self.thoughts = thoughts


    #FUNCTIONS
    def get_impotrance_score(self):
        '''
        create importance score (with a range of 0 to 8)
        if it is prioritized (is_prioritized==True), give it a score of 8 (top score)
        if not, consider the importance:
            if it is date dependent, give it a score of 7 - (days in between)
            (this means if it is due in less than 1 day, it will have a score of 7)
            if it is super important, give it a score 6.5 (while it will still be less important than those due in less than 1 day)
            if it is kinda important, give it a score 3.5 (date dependent will result higher if it is due in three days)
        '''
        if self.is_prioritized:
            score = 8
        elif self.importance==Importance.SUPER_IMPORTANT:
            score = 6.5
        elif self.importance==Importance.KINDA_IMPORTANT:
            score = 3.5
        elif self.importance==Importance.DATE_DEPENDENT:
            time_left = self.date_due - datetime.now()
            days_left = time_left.days
            score = max(0,7-days_left)
        else: # if == None
            score = 0
        return score 
    
    def update_labels(self,labels):
        '''
        if the added instance has a new label, append it to the list of labels
        if it is already existed, ignore this
        '''
        if self.label not in labels:
            labels.append(self.label)

    # def get_current_labels(self,labels):
    #     return ...


In [48]:
# examples
task1 = Task('Complete assignment', label='school', date_due=datetime(2023, 11, 21, 23, 59), importance='SUPER_IMPORTANT', 
             is_prioritized=False, is_overdue=False, if_overdue='MARK_RED_PRIORITIZE', details='30% of the final grade!', 
             steps='1. intro; 2. body; 3. conclusion.', thoughts='this is so hard omg')

task2 = Task('Pick up stones', label='life', importance='KINDA_IMPORTANT', steps='find a stone & pick it up.', thoughts='stones are cool.')

task3 = Task('hackathon', label='school', date_due=datetime(2023, 11, 19, 23, 59), importance='DATE_DEPENDENT', if_overdue='NEVER_MIND',
             details='I like coding.', steps='1. this object; 2. create functions; 3. show it as a dataframe.', thoughts='yessir')

In [49]:
# tests
def print_task_details(task):
    for attr, value in task.__dict__.items():
        print(f"{attr}: {value}")


print_task_details(task1)
print(task1.get_impotrance_score())




task: Complete assignment
is_done: False
label: school
date_added: 2023-11-19 16:59:06.958980
date_due: 2023-11-21 23:59:00
importance: Importance.SUPER_IMPORTANT
is_prioritized: False
is_overdue: False
if_overdue: IfOverdue.MARK_RED_PRIORITIZE
details: 30% of the final grade!
steps: 1. intro; 2. body; 3. conclusion.
thoughts: this is so hard omg
6.5


# PRESENTING THE DATAFRAME

### BACKEND DATAFRAME

In [50]:
cols_back = ['task','done?','label','date added','date due',
        'importance','prioritize it?','is it overdue?','if is overdue','details',
        'steps','thoughts','importance score']
df_back = pd.DataFrame(
    columns = cols_back
)
print(df_back.shape)
df_back

(0, 13)


,task,done?,label,date added,date due,importance,prioritize it?,is it overdue?,if is overdue,details,steps,thoughts,importance score


### FRONTEND DATAFRAME

In [51]:
cols_front = ['done?','task','label','date added','date due','importance','prirotize it?']
df_front = pd.DataFrame(
    columns=cols_front
)
print(df_front.shape)
df_front

(0, 7)


,done?,task,label,date added,date due,importance,prirotize it?


In [52]:
# link a Task() instance to df
def get_instance_info(task):
    task_values = []
    for attr, value in task.__dict__.items():
        if isinstance(value, Enum):
            value = value.value
            # print(value)
        task_values.append(value)
    task_values.append(task.get_importance_score)
    return task_values

def add_instance_to_df(df, task):
    task_values = get_instance_info(task)
    df.loc[len(df)] = task_values
    return df


In [53]:
# tests
print(get_instance_info(task1))

df_try = df_back.copy()
add_instance_to_df(df_try,task1)
add_instance_to_df(df_try,task3)
df_try.drop_duplicates(inplace=True)
df_try

AttributeError: 'Task' object has no attribute 'get_importance_score'

In [24]:
# create importance score 
# use ordinal encoder?

def create_importance_score(df):
    '''
    create importance score (with a range of 0 to 7)
    if it is date dependent, give it a score of 7 - (days in between)
    (this means if it is due in less than 1 day, it will have a score of 7)
    if it is super important, give it a score 6.5 (while it will still be less important than those due in less than 1 day)
    if it is kinda important, give it a score 3.5 (date dependent will result higher if it is due in three days)
    '''
    importance_score = []
    for index, row in df.iterrows():
        if row['importance']=='Date Dependent':
            time_left = row['date due'] - datetime.now()
            days_left = time_left.days
            score = max(0,7-days_left)
            importance_score.append(score)
        elif row['importance']=='Super Important':
            importance_score.append(6.5)
        elif row['importance']=='Kinda Important':
            importance_score.append(3.5)

    df_scored = df.copy()

    df_scored['importance_score']=importance_score
    return df_scored


In [27]:
df_try_scored = create_importance_score(df_try)

print(df_try_scored.shape)
df_try_scored

(2, 13)


,task,done?,label,date added,date due,importance,prioritize it?,is it overdue?,if is overdue,details,steps,thoughts,importance_score
0,Complete assignment,False,school,2023-11-19 14:34:29.097861,2023-11-21 23:59:00,Super Important,False,False,Mark Red and Prioritize,30% of the final grade!,1. intro; 2. body; 3. conclusion.,this is so hard omg,6.5
1,hackathon,False,school,2023-11-19 14:34:29.097921,2023-11-19 23:59:00,Date Dependent,False,False,Never Mind,I like coding.,1. this object; 2. create functions; 3. show i...,yessir,7.0


In [28]:
print(df.shape)
df

(0, 12)


,task,done?,label,date added,date due,importance,prioritize it?,is it overdue?,if is overdue,details,steps,thoughts


In [13]:
df

,task,done?,label,date added,date due,importance,prioritize it?,is it overdue?,if is overdue,details,steps,thoughts
0,Complete assignment,False,school,2023-11-19 14:34:29.097861,2023-11-21 23:59:00,Super Important,False,False,Mark Red and Prioritize,30% of the final grade!,1. intro; 2. body; 3. conclusion.,this is so hard omg
1,hackathon,False,school,2023-11-19 14:34:29.097921,2023-11-19 23:59:00,Date Dependent,False,False,Never Mind,I like coding.,1. this object; 2. create functions; 3. show i...,yessir


# let the user input their data

In [ ]:
# ...
app = Flask(__name__)

@app.route('/submit', methods=['POST'])


# ARCHIVE